# PANDAS TIPS & TRICS

In [34]:
import plotly.express as px
import pandas as pd
from pathlib import Path

aer_file_filter = 'Place-POL_Mid-To-Satellite-POLLEO'
aer_folder = Path ( 'AER' )

df = pd.DataFrame ()  # Inicjalizacja głównego DataFrame, który będzie zawierał wszystkie dane
for child in aer_folder.iterdir () :
    if child.is_file () and child.suffix == '.csv' and child.name.startswith ( aer_file_filter ) :
        print ( child.name )
        df_ael = pd.read_csv ( child , on_bad_lines = 'skip' , delimiter = ',' , header = 0 , names = [ 'UTC' , 'A' , 'E' , 'R' ] )
        df_ael [ 'To satellite' ] = child .stem .split ( '-' ) [ -1 ] .split ( '_' ) [0]
        df_ael [ 'From terminal' ] = child .stem .split ( '-' ) [ 1 ] .split ( '-' ) [0]

        # Convert values, forcing errors to NaN
        df_ael [ 'UTC' ] = pd.to_datetime ( df_ael [ 'UTC' ] , errors = 'coerce' )
        df_ael [ 'A' ] = pd.to_numeric ( df_ael [ 'A' ] , errors = 'coerce' )
        df_ael [ 'E' ] = pd.to_numeric ( df_ael [ 'E' ] , errors = 'coerce' )
        df_ael [ 'R' ] = pd.to_numeric ( df_ael [ 'R' ] , errors = 'coerce' )

        # Drop rows where value is NaN
        #df_ael = df_ael.dropna ( subset = ['Time (UTCG)'] )
        #df_ael = df_ael.dropna ( subset = ['Azimuth (deg)'] )
        #df_ael = df_ael.dropna ( subset = ['Elevation (deg)'] )
        #df_ael = df_ael.dropna ( subset = ['Range (km)'] )
        # Usuwanie wierszy z wartościami NaN w interesujących nas kolumnach
        df_ael.dropna ( subset = [ 'UTC' , 'A' , 'E' , 'R' ] , inplace = True )

        # Dodawanie danych z aktualnego pliku do głównego DataFrame
        df = pd.concat ( [ df , df_ael ] , ignore_index = True )
df.info ()
df.shape

Place-POL_Mid-To-Satellite-POLLEO11_AER.csv
Place-POL_Mid-To-Satellite-POLLEO12_AER.csv
Place-POL_Mid-To-Satellite-POLLEO21_AER.csv
Place-POL_Mid-To-Satellite-POLLEO22_AER.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1717474 entries, 0 to 1717473
Data columns (total 6 columns):
 #   Column         Dtype         
---  ------         -----         
 0   UTC            datetime64[ns]
 1   A              float64       
 2   E              float64       
 3   R              float64       
 4   To satellite   object        
 5   From terminal  object        
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 78.6+ MB


(1717474, 6)

In [35]:
df .head ( 2 )

,UTC,A,E,R,To satellite,From terminal
0,2000-01-01 17:25:29.241,193.102,10.000,2772.447383,POLLEO11,POL_Mid
1,2000-01-01 17:25:30.000,193.067,10.056,2768.072502,POLLEO11,POL_Mid


In [36]:
df .tail ( 2 )

,UTC,A,E,R,To satellite,From terminal
1717472,2000-04-01 08:23:22.000,171.622,10.028,2770.063784,POLLEO22,POL_Mid
1717473,2000-04-01 08:23:22.380,171.601,10.000,2772.210683,POLLEO22,POL_Mid


In [37]:
df [ 'To satellite' ] .head ( 2 )

0    POLLEO11
1    POLLEO11
Name: To satellite, dtype: object

`.loc[]` - access a group of rows and columns by label(s) or a boolean array. It is primarily label based, but may also be used with a boolean array.

In [39]:
df .loc [ 1_717_472 ]

UTC              2000-04-01 08:23:22
A                            171.622
E                             10.028
R                        2770.063784
To satellite                POLLEO22
From terminal                POL_Mid
Name: 1717472, dtype: object

## PANDAS FILTERING AND ORDERING

In [41]:
df [ df [ 'E' ] > 89.8 ]

,UTC,A,E,R,To satellite,From terminal
565861,2000-01-30 11:36:11,26.173,89.913,1013.270613,POLLEO12,POL_Mid
1097812,2000-02-21 09:47:35,6.199,89.825,1013.278454,POLLEO21,POL_Mid
1625351,2000-03-13 04:49:08,138.815,89.891,1013.263201,POLLEO22,POL_Mid


In [42]:
plane1 = [ 'POLLEO11' , 'POLLEO12' ,'POLLEO13' ,'POLLEO14' ]
df [ df [ 'To satellite' ] .isin ( plane1 ) ]

,UTC,A,E,R,To satellite,From terminal
0,2000-01-01 17:25:29.241,193.102,10.000,2772.447383,POLLEO11,POL_Mid
1,2000-01-01 17:25:30.000,193.067,10.056,2768.072502,POLLEO11,POL_Mid
2,2000-01-01 17:25:31.000,193.021,10.131,2762.311517,POLLEO11,POL_Mid
3,2000-01-01 17:25:32.000,192.976,10.205,2756.553412,POLLEO11,POL_Mid
4,2000-01-01 17:25:33.000,192.930,10.280,2750.798209,POLLEO11,POL_Mid
...,...,...,...,...,...,...
858750,2000-04-01 04:52:57.000,168.100,10.284,2750.422531,POLLEO12,POL_Mid
858751,2000-04-01 04:52:58.000,168.052,10.210,2756.154352,POLLEO12,POL_Mid
858752,2000-04-01 04:52:59.000,168.004,10.135,2761.889180,POLLEO12,POL_Mid
858753,2000-04-01 04:53:00.000,167.957,10.061,2767.626992,POLLEO12,POL_Mid


In [43]:
df [ df [ 'To satellite' ] .str.contains ( 'POLLEO1' ) ]

,UTC,A,E,R,To satellite,From terminal
0,2000-01-01 17:25:29.241,193.102,10.000,2772.447383,POLLEO11,POL_Mid
1,2000-01-01 17:25:30.000,193.067,10.056,2768.072502,POLLEO11,POL_Mid
2,2000-01-01 17:25:31.000,193.021,10.131,2762.311517,POLLEO11,POL_Mid
3,2000-01-01 17:25:32.000,192.976,10.205,2756.553412,POLLEO11,POL_Mid
4,2000-01-01 17:25:33.000,192.930,10.280,2750.798209,POLLEO11,POL_Mid
...,...,...,...,...,...,...
858750,2000-04-01 04:52:57.000,168.100,10.284,2750.422531,POLLEO12,POL_Mid
858751,2000-04-01 04:52:58.000,168.052,10.210,2756.154352,POLLEO12,POL_Mid
858752,2000-04-01 04:52:59.000,168.004,10.135,2761.889180,POLLEO12,POL_Mid
858753,2000-04-01 04:53:00.000,167.957,10.061,2767.626992,POLLEO12,POL_Mid
